## 数据规整化： 清理、转换、合并、重塑

### 合并数据

pandas对象中的数据可以通过一些内置的方式进行合并
- pandas.merge 可以根据一个或者多个健将不同DataFrame中的行连接起来
- pandas.concat 可以沿着一个轴将多个对象堆叠到一起
- combine_first 重复数据编接，用一个对象中的值填充另一个对象中的缺失值

#### 数据库风格的DataFrame合并
- merge的参数
    - left 左df
    - right 右df
    - on 连接列，未指定则是交集
    - left_on 左侧连接列
    - right_on 右侧连接列
    - left_index 左侧的行索引用作连接健
    - right_index 右...
    - sort 根据连接健排序，默认是True
    - suffixes 字符串值元组
    - copy 默认总是复制，除非设置为False
    

In [ ]:
from pandas import DataFrame, Series
df1 = DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'], 
                 'data1': range(7)})

df2 = DataFrame({'key': ['a', 'b', 'd'], 
                 'data2': range(3)})

print ( df1 ) ;print( '-'*32 ) 
print ( df2 ) ; print( '-'*32 )
print ( pd.merge(df1, df2)) ; print( '-'*32 )
print ( pd.merge(df1, df2, on='key') )

In [ ]:
df3 = DataFrame({'lkey': ['b', 'b', 'a', 'c', 'a', 'a', 'b'], 
                 'data1': range(7)})

df4 = DataFrame({'rkey': ['a', 'b', 'd'], 
                 'data2': range(3)})

pd.merge(df3, df4, left_on='lkey', right_on='rkey')


In [ ]:
print ( pd.merge(df1, df2, how='inner') ) ; print('-'*32)
print ( pd.merge(df1, df2, how='outer') ) ; print('-'*32)
print ( pd.merge(df1, df2, how='left') )  ; print('-'*32)
print ( pd.merge(df1, df2, how='right') ) ; print('-'*32)

- 多对多的合并操作非常简单，无需额外的工作
- 多对多连接产生的结果是行的**笛卡尔积**

In [ ]:
df1 = DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'], 
                 'data1': range(6)})
df2 = DataFrame({'key': ['a', 'b', 'a', 'b', 'd'], 
                 'data2': range(5)})

print ( df1 ) ;print('-'*32)
print ( df2 ) ;print('-'*32)
print ( pd.merge(df1, df2, on='key', how='left') )

需要根据多个健进行合并，传入一个由列名组成的列表即可

In [ ]:
left = DataFrame({'key1': ['foo', 'foo', 'bar'], 
                  'key2': ['one', 'two', 'one'],
                  'lval': [1, 2, 3]})

right = DataFrame({'key1': ['foo', 'foo', 'bar', 'bar'], 
                   'key2': ['one', 'one', 'one', 'two'],
                   'rval': [4, 5, 6, 7]})

print ( left )
print ( right )
pd.merge(left, right, on=['key1', 'key2'], how='outer')

In [ ]:
pd.merge(left, right, on='key1')

In [ ]:
pd.merge(left, right, on='key1', suffixes=('_left', '_right'))

#### 索引上的合并
    有时候，DataFrame中的连接健位于其索引中。
    传入left_index=True或right_index=True以说明索引应该被用作连接健

In [ ]:
import pandas as pd
from pandas import DataFrame, Series
import numpy as np
left1 = DataFrame(
    {
        'key': list('abaabc'),
        'value':range(6)
    }
)

right1 = DataFrame(
    {
        'group_val': [3.5, 7]
    },
    index=['a','b']
)

print(left1) ; print('-'*32)
print(right1) ; print('-'*32)

d = pd.merge(left1, right1,left_on='key', 
         right_index=True)

print( d ) ; print('-'*32)

d = pd.merge(left1, right1,left_on='key', 
         right_index=True, how='outer')
print( d ) ; print('-'*32)


**层次化索引**

In [ ]:
lefth = DataFrame(
    {
    'key1':['Ohio', 'Ohio', 'Ohio', 'Nevada', 'Nevada'],
    'key2':[2000,2001,2002,2001,2002],
    'data':np.arange(5)
    }
)

righth = DataFrame(
    np.arange(12).reshape((6,2)),
    index=[
        ['Nevada', 'Nevada','Ohio','Ohio','Ohio','Ohio'],
        [2001,2000,2000,2000,2001,2002]
          ],
    columns=['event1', 'event2']
)
print( lefth )
print( righth )

d = pd.merge( lefth,righth,left_on=['key1','key2'], 
        right_index=True)
print(d) ;print('-'*32)

d = pd.merge( lefth,righth,left_on=['key1','key2'], 
        right_index=True, how='outer')
print(d) ;print('-'*32)



In [ ]:
left2 = DataFrame(
    np.arange(1,7).reshape((3,2)),
    index=list('ace'),
    columns=['Ohio','Nevada']
)
right2 = DataFrame(
    np.arange(7,15).reshape(4,2),
    index=list('bcde'),
    columns=['Missouri','Alabama']
)

print( left2 ) ; print( '-'*32 )
print( right2 ) ; print( '-'*32 )
pd.merge(left2, right2, how='outer', 
         left_index=True, right_index=True)

**DataFrame还有一个join实例方法，它能更为方便地实现按索引合并**

In [ ]:
left2.join(right2, how='outer')

In [ ]:
print(left1);print('-'*32)
print(right1);print('-'*32)
left1.join(right1,on='key')

In [ ]:
another = DataFrame(
    np.arange(7,15).reshape((4,2)),
    index=list('acef'),
    columns=['New York', 'Oregon']
)
d1=left2.join([right2, another])
d2=left2.join([right2, another], how='outer')
[d1,d2]

#### 轴向连接
另一种数据合并运算也被称作连接(concatenation),绑定(binding),堆叠（stacking）

In [ ]:
arr = np.arange(12).reshape((3,4))
r = np.concatenate([arr,arr], axis=0)
print( r ) ; print('-'*32)
r = np.concatenate([arr,arr], axis=1)
print( r ) ; print('-'*32)

In [ ]:
s1 = Series([0,1], index=['a','b'])
s2 = Series([2,3,4],index=['c','d','e'] )
s3 = Series([5,6], index=['f','g'])

d=pd.concat([s1,s2,s3],axis=0)
print( d ) ;print('-'*32)

d=pd.concat([s1,s2,s3], axis=1,sort=True)
print( d ) ; print('-'*32)

In [ ]:
s4 = pd.concat([s1 * 5, s3])

r = pd.concat([s1,s4], axis=1,sort=True)
print( r ) ; print('-'*32)

r = pd.concat([s1, s4], axis=1, join='inner',sort=True)
print( r ) ; print('-'*32)

**可以通过join_axes指定要在其他轴上使用的索引**

In [ ]:
pd.concat([s1,s4], axis=1,
         join_axes=[['a','c','b','e']],
          sort=False
         )

In [ ]:
result = pd.concat([s1,s2,s3], 
                   keys=['one','two','three']
                  )
result

In [ ]:

result.unstack

如果沿着axis=1对Series进行合并，则keys就会成为DataFrame的列头

In [ ]:
result = pd.concat([s1,s2,s3], 
                   keys=['one','two','three'],
                   axis=1,sort=False
                  )

result

In [ ]:
df1 = DataFrame(
    np.arange(6).reshape((3,2)),
    index=list('abd'),
    columns=['one','two']
)

df2 = DataFrame(
    np.arange(4).reshape((2,2))+5,
    index=list('ac'),
    columns=['three', 'four']
)

d = pd.concat([df1,df2], 
          axis=0,sort=True,
          keys=['level1', 'level2']
         )
print( d ) ; print('-'*32)

d = pd.concat([df1,df2], 
          axis=1,sort=True,
          keys=['level1', 'level2']
         )
print( d ) ; print('-'*32)

***如果传入的不是列表，而是字典，那么字典的健会被当作keys选项的值***

In [ ]:
pd.concat(
    {
        'level1':df1,
        'level2':df2
    },
    axis=1,sort=False
)

**ignore_index=True**

In [ ]:
df1 = DataFrame(np.random.randn(3,4),
                columns=list('abcd')
               )
df2 = DataFrame(np.random.randn(2,3),
                columns=list('bda')
               )
pd.concat([df1,df2], ignore_index=True, sort=False)

**concat函数的参数**
- objs  列表或者字典
- axis  指明连接的轴向
- join inner/outer 默认是outer，索引是按照交集还是并集合并
- join_axes 指明用于其他n-1条轴上的索引 
- keys 与objs有关的值
- levels 指定用于层次化索引各级别上的索引
- names 用于创建levels上的名称结合上面两项使用
- verify_integrity 检查结果对象新轴上的重复情况
- ignore_index 不保留连接轴上的索引

#### 合并重叠数据
索引全部或者部分重叠的两个数据集，并且给一点点启发性

In [ ]:
a = Series(
    [np.nan,2.5,np.nan,3.5,4.5,np.nan],
    index=list('fedcba')
)

b = Series(
    np.arange(len(a),dtype=np.float64),
    index=list('fedcba')
)



In [ ]:
a

In [ ]:
b

In [ ]:
np.where(pd.isnull(a),b,a)

In [ ]:
b[-1] = np.nan
b

In [ ]:
b[:-2].combine_first(a[2:])

对于DataFrame，combine_first自然也会在列上做同样的事情

In [ ]:
df1 =DataFrame(
    {
        'a':[1.,np.nan,5,np.nan],
        'b':[np.nan,2,np.nan,6],
        'c':range(2,18,4)
    }
)

df2 = DataFrame(
    {
        'a':[5.,4,np.nan,3,7],
        'b':[np.nan,3,4,6,8]
    }
)
df1.combine_first(df2)

### 重塑和轴向旋转
- reshape
- pivot


##### 重塑层次化索引
- stack 列 旋转成  行
- unstack 行 旋转成 列

In [ ]:
data = DataFrame(
    np.arange(6).reshape((2,3)),
    index=pd.Index(['Ohio','Colorado'], name='state'),
    columns=pd.Index(['one','two','three'],name='number')
)

data


In [ ]:
result = data.stack()
result

In [ ]:
result.unstack()

**默认情况下，stack/unstack操作的是最内层**
传入分层level的编号或名称即可以对其进行操作


In [ ]:
result.unstack(0)

In [ ]:
result.unstack('state')

In [ ]:
s1 = Series([0,1,2,3], index=['a','b','c','d'])
s2 = Series([4,5,6], index=['c','d','e'])
data2 = pd.concat([s1,s2],keys=['one','two'])
data2

In [ ]:
data2.unstack()

In [ ]:
data2.unstack().stack()

In [ ]:
data2.unstack().stack(dropna=False)

在对DataFrame进行unstack操作时，作为旋转轴的级别将会成为结果中的最低级别

In [ ]:
df = DataFrame(
    {
        'left':result,
        'right':result +5
    },
    columns=pd.Index(['lift','right'],name='side')
)

df



In [ ]:
df.unstack('state')

In [ ]:
df.unstack('state').stack('side')

#### 将长格式转换为宽格式

时间序列数据通常是以所谓的“长格式”或“堆叠格式”存储在数据库和CSV中
- 长格式 long
- 堆叠格式 stack


#### XXXXX

In [ ]:
ldata[:10]

In [ ]:
pivoted = ldata.pivot('date', 'item', 'value')
pivoted.head()

In [ ]:
ldata['values2'] = np.random.randn(len(ldata))
ldata[:10]

In [ ]:
pivoted = ldata.pivot('date','item')
pivoted[:5]

In [ ]:
pivoted['value'][:5]

In [ ]:
unstacked = ldata.set_index(['date', 'item'] ).unstack('item')
unstacked[:7]

### 数据转化

#### 移除重复数据

In [ ]:
data = DataFrame(
    {
        'k1':['one']*3+['two']*4,
        'k2':[1,1,2,3,3,4,4]
    }
)
data

In [ ]:
data.duplicated()

In [ ]:
data.drop_duplicates()

In [ ]:
data['v1'] = range(7)
data.drop_duplicates(['k1'])

#### 利用函数或映射进行数据转换

In [ ]:
data = DataFrame({'food': ['bacon', 'pulled pork', 'bacon', 'Pastrami', 
                           'corned beef', 'Bacon', 'pastrami', 'honey ham',
                           'nova lox'],
                  'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
data

In [ ]:
meat_to_animal = { 
    'bacon': 'pig', 
    'pulled pork': 'pig', 
    'pastrami': 'cow', 
    'corned beef': 'cow', 
    'honey ham': 'pig', 
    'nova lox': 'salmon'
}
data['food'].map(str.lower).map(meat_to_animal)

In [ ]:
data['food'].map(lambda x: meat_to_animal[x.lower()])

### 替换值
- fillna
- map
- replace

In [ ]:
data = Series(
    [1.,-999,2,-999,-1000,3]
)
print ( data )

#1 -> 1
print ( data.replace(-999,np.nan) )


print ( data.replace( [-999, -1000] ,np.nan) )


#list -> list
print ( data.replace( [-999, -1000], [np.nan,0] ))

#dict 
print ( data.replace( {-999: np.nan, -1000:0} ))

#### 重命名轴索引
**轴标签也可以通过函数或映射进行转换**
- rename
- map

In [ ]:
data = DataFrame(np.arange(12).reshape((3, 4)),
                 index=['Ohio', 'Colorado', 'New York'],
                 columns=['one', 'two', 'three', 'four'])
data

In [ ]:
data.index
data.index.map(str.upper)

In [ ]:
data.rename(index=str.title, columns=str.upper)

In [ ]:
data.rename(index={'OHIO':'INDIANA'},inplace=True)
data

#### 离散化和面元划分

In [ ]:
ages = [20,22,25,27,21,23,37,31,61,45,41,32]
bins = [18,25,35,60,100]
cats = pd.cut(ages, bins)
cats

In [ ]:
#cats.label
#cats.levels
cats.value_counts

In [ ]:
group_names = ['Youth', 'YoungAdult', 
               'MiddleAged', 'Senior']
pd.cut(ages, bins, labels=group_names)

In [ ]:
data = np.random.randn(1000)
cats = pd.qcut(data,4)
cats

In [ ]:
pd.value_counts(cats)

In [ ]:
pd.qcut(data, [0,0.1,0.5,0.9,1.0])

#### 检验和过滤异常值

In [ ]:
np.random.seed(12345)
data = DataFrame(np.random.randn(1000,4))
data.describe()

In [ ]:
col = data[3]
col[np.abs(col) > 3]

In [ ]:
data[(np.abs(data) > 3).any(1)]

In [ ]:
data[np.abs(data) > 3] = np.sign(data)*3
data.describe()

#### 排列和随机采样

In [ ]:
df = DataFrame(np.arange(5*4).reshape(5,4))
sampler = np.random.permutation(5)
sampler

In [ ]:
df

In [ ]:
df.take(sampler)

In [ ]:
df.take(np.random.permutation(len(df))[:3])

In [ ]:
bag = np.array([5,7,-1,6,4])
sampler = np.random.randint(0,len(bag),size=10)
sampler

In [ ]:
draws = bag.take(sampler)
draws

#### 计算指标/哑变量

In [ ]:
df = DataFrame(
    {
        'key':list('bbacab'),
        'data1':range(6)
    }
)
print( df )
pd.get_dummies(df['key'])

In [ ]:
dummies = pd.get_dummies(df['key'], prefix='key')
dummies

In [ ]:
df_with_dummy = df[['data1']].join(dummies)
df_with_dummy

In [ ]:
mnames = ['movie_id','title','genres']


In [ ]:
movies = pd.read_csv('pydata/ch02/movielens/movies.dat',
                     sep= '::',
                     header= None,
                     names= mnames
                    )
movies[:10]

In [ ]:
genre_iter = (set(x.split('|')) for x in movies.genres)

In [ ]:
genres = sorted( set.union(*genre_iter) )
genres

In [ ]:
dummies = DataFrame(np.zeros((len(movies), len(genres))), 
                    columns=genres)

In [ ]:
for i,gen in enumerate(movies.genres):
    dummies.loc[i,gen.split('|')] = 1

In [ ]:
movies_windic = movies.join(dummies.add_prefix('Genre_'))
movies_windic.iloc[0]

In [ ]:
values = np.random.rand(10)
values

In [ ]:
bins = [0,0.2,0.4,0.6,0.8,1]
pd.get_dummies(pd.cut(values, bins))

### 字符串操作

#### 字符串对象方法

In [ ]:
val = 'a,b, guido'
val.split(',')

In [ ]:
pieces = [x.strip() for x in val.split(',') ]
pieces

In [ ]:
first, second, third = pieces
first+'::' + second + '::' + third

In [ ]:
'guido' in val

In [ ]:
val.index(',')

In [ ]:
val.find(':')

In [ ]:
val.count(',')

In [ ]:
val.replace(',', '::')

In [ ]:
val.replace(',','')

**Python内置的字符串方法**
- count
- endwith, startwith
- join
- index
- find
- rfind
- replace
- strip, rstrip, lstrip
- split
- lower, upper
- ljust, rjust

#### 正则表达式
re模块的函数可以分为三个大类
- 模式匹配
- 替换
- 拆分

In [ ]:
import re
text = "foo    bar\t baz.   \tqux"
re.split('\s+', text)

In [ ]:
regex = re.compile('\s+')
regex.split(text)

In [ ]:
regex.findall(text)

In [ ]:
text = """Dave dave@google.com Steve 
steve@gmail.com
Rob rob@gmail.com
Ryan ryan@yahoo.com
"""
pattern = r'[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,4}'
# re.IGNORECASE makes the regex case-insensitive 
regex = re.compile(pattern, flags=re.IGNORECASE)

In [ ]:
regex.findall(text)

In [ ]:
m = regex.search(text)
m

In [ ]:
text[m.start():m.end()]

In [ ]:
print(regex.match(text))
print(regex.sub('REDACTED',text))

P232 
In [235]

**正则表达式方法**
- findall,finditer
- match
- search
- split
- sub, subn


#### pandas中矢量化的字符串函数

In [ ]:
data = {'Dave': 'dave@google.com', 'Steve': 'steve@gmail.com', 
        'Rob': 'rob@gmail.com', 
        'Wes': np.nan}

In [ ]:
data = Series(data)
data

In [ ]:
data.isnull()

In [ ]:
data.str.contains('gmail')

In [ ]:
pattern

In [ ]:
 data.str.findall(pattern, flags=re.IGNORECASE)

In [ ]:
matches = data.str.match(pattern, flags=re.IGNORECASE)
matches

In [ ]:
matches.str.get(1)

In [ ]:
matches.str[0]

In [ ]:
data.str[:5]

**矢量化的字符串方法**
- cat
- contains
- count
- endswith, startswith
- findall
- get
- join
- len
- lower, upper
- match
- pad
- center
- repeat
- replace
- slice
- split
- strip, rstrip, lstrip

### 示例： USDA视频数据库

In [ ]:
import json
db = json.load(open('pydata/ch07/foods-2011-10-03.json'))
len(db)

In [ ]:
db[0].keys()

In [ ]:
db[0]['nutrients'][0]

In [ ]:
nutrients = DataFrame(db[0]['nutrients'])
nutrients[:7]

In [ ]:
info_keys = ['description', 'group','id','manufacturer']

In [ ]:
info = DataFrame(db, columns=info_keys)

In [ ]:
info[:5]

In [ ]:
info.describe()

In [ ]:
pd.value_counts(info.group)[:10]

In [ ]:
nutrients = []
for rec in db:
    fnuts = DataFrame(rec['nutrients']) 
    fnuts['id'] = rec['id'] 
    nutrients.append(fnuts)
    
nutrients = pd.concat(nutrients, ignore_index=True)
    
nutrients.head()

In [ ]:
nutrients.duplicated().sum()

In [ ]:
nutrients = nutrients.drop_duplicates()

In [ ]:
nutrients.head()

In [ ]:
col_mapping = {'description': 'food',
               'group': 'fgroup'
              }
info = info.rename(columns=col_mapping, copy=False )
info.head()

In [ ]:
col_mapping = {
    'description':'nutrient',
    'group': 'nutgroup'
}
nutrients = nutrients.rename( columns=col_mapping, copy=False)
nutrients.head()

In [ ]:
ndata = pd.merge(nutrients, info, on='id', how='outer')
ndata.head()

In [ ]:
ndata.loc[30000]

In [ ]:
result = ndata.groupby(['nutrient', 'fgroup'])['value'].quantile(0)

In [ ]:
result['Zinc, Zn'].sort_values().plot(kind='barh')

In [ ]:
by_nutrient = ndata.groupby(['nutgroup', 'nutrient'])

get_maximum = lambda x: x.xs(x.value.idxmax())
get_minimum = lambda x: x.xs(x.value.idxmin())

max_foods = by_nutrient.apply(get_maximum)[['value','food']]

max_foods.food = max_foods.food.str[:50]

In [ ]:
max_foods.loc['Amino Acids']['food'].head()